<a href="https://colab.research.google.com/github/drshahizan/special-topic-data-engineering/blob/main/project/submission/MichelinStar/MichelinStar_Project_TMDB_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

######**Group Members:**
######Tan Yong Sheng (A20EC0157)
######Yong Zhi Yan (A20EC0172)
######Eddie Wong Chung Pheng	(A20EC0031)
######Low Junyi (A20EC0071)
######Vincent Boo Ee Khai(A20EC0231)

# **Web Scraping Using API**

The code below demonstrate the way we retrieve the data from an online website called [TMDB](https://www.themoviedb.org). It is the most popular and editable database related to movie. The API used is given from TMDB where we need to register first and request from the website.

In [1]:
from google.colab import files
import pandas as pd
import requests

response = requests.get('https://api.themoviedb.org/3/movie/top_rated?api_key=56db60834991ae2a9aca81812b1974c5&language=en-US&page=1')
df = pd.DataFrame()  # Creating a DataFrame

if response.status_code == 200:
    for i in range(1, 30):
        response = requests.get('https://api.themoviedb.org/3/movie/top_rated?api_key=56db60834991ae2a9aca81812b1974c5&language=en-US&page={}'.format(i))
        results = response.json()['results']
        temp_df = pd.DataFrame(results)[['id', 'title', 'overview', 'release_date', 'popularity', 'vote_average', 'vote_count']]

        # Add a shorter overview column
        temp_df['short_overview'] = temp_df['overview'].str[:70]

        # Retrieve additional data for each movie
        for index, row in temp_df.iterrows():
            movie_id = row['id']
            movie_response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key=56db60834991ae2a9aca81812b1974c5&language=en-US')
            movie_data = movie_response.json()

            # Extract genres (first result only)
            genres = [genre['name'] for genre in movie_data['genres']]
            temp_df.at[index, 'genre'] = genres[0] if genres else 'Unknown'

            # Extract runtime
            temp_df.at[index, 'runtime'] = int(movie_data['runtime']) * 60

            # Extract country (first result only)
            production_countries = [country['name'] for country in movie_data['production_countries']]
            temp_df.at[index, 'country'] = production_countries[0] if production_countries else 'Unknown'

            # Extract actors
            credits_response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key=56db60834991ae2a9aca81812b1974c5')
            credits_data = credits_response.json()
            actors = [actor['name'] for actor in credits_data['cast'][:3]]  # Retrieve top 3 actors
            temp_df.at[index, 'actors'] = ', '.join(actors)

            # Extract directors (first result only)
            credits_response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key=56db60834991ae2a9aca81812b1974c5')
            credits_data = credits_response.json()
            directors = [crew['name'] for crew in credits_data['crew'] if crew['job'] == 'Director']
            temp_df.at[index, 'director'] = directors[0] if directors else 'Unknown'

            # Extract movie link
            temp_df.at[index, 'movie_link'] = movie_data['homepage'] if 'homepage' in movie_data else 'Unknown'

            # Extract trailer link
            videos_response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}/videos?api_key=56db60834991ae2a9aca81812b1974c5')
            videos_data = videos_response.json()
            trailers = [video['key'] for video in videos_data['results'] if video['type'] == 'Trailer']
            if trailers:
                temp_df.at[index, 'trailer_link'] = f"https://www.youtube.com/watch?v={trailers[0]}"

        # Add default values for new columns
        temp_df['feature'] = 1
        temp_df['kid_restriction'] = 0

        df = df.append(temp_df, ignore_index=True)
else:
    print('Error', response.status_code)

df['rating'] = 0  # Initialize the 'rating' column with default value 0

df.loc[df['vote_average'] > 8.3, 'rating'] = 5
df.loc[(df['vote_average'] > 8.0) & (df['vote_average'] <= 8.3), 'rating'] = 4
df.loc[(df['vote_average'] > 7.7) & (df['vote_average'] <= 8.0), 'rating'] = 3
df.loc[(df['vote_average'] > 7.4) & (df['vote_average'] <= 7.7), 'rating'] = 2
df.loc[(df['vote_average'] > 7.0) & (df['vote_average'] <= 7.4), 'rating'] = 1

# Extract year from release_date
df['release_year'] = pd.to_datetime(df['release_date']).dt.year

print(df.shape)
print(df.head(5))

# Save the DataFrame as a CSV file
df.to_csv('movie_top_rated.csv', index=False)

<ipython-input-1-97ce71f64688>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
<ipython-input-1-97ce71f64688>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
<ipython-input-1-97ce71f64688>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
<ipython-input-1-97ce71f64688>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)
<ipython-input-1-97ce71f64688>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pan

(580, 19)
      id                        title  \
0    238                The Godfather   
1    278     The Shawshank Redemption   
2    240        The Godfather Part II   
3  19404  Dilwale Dulhania Le Jayenge   
4    424             Schindler's List   

                                            overview release_date  popularity  \
0  Spanning the years 1945 to 1955, a chronicle o...   1972-03-14     110.936   
1  Framed in the 1940s for the double murder of h...   1994-09-23      90.699   
2  In the continuing saga of the Corleone crime f...   1974-12-20      52.110   
3  Raj is a rich, carefree, happy-go-lucky second...   1995-10-20      38.611   
4  The true story of how businessman Oskar Schind...   1993-12-15      46.884   

   vote_average  vote_count  \
0           8.7       18208   
1           8.7       24094   
2           8.6       10984   
3           8.6        4163   
4           8.6       14240   

                                      short_overview   genre  runtime

<ipython-input-1-97ce71f64688>:60: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df, ignore_index=True)


# Save as CSV

Now we have successfully retrieve the data from TMDB. We proceed to choose the required column and save it into csv file for further import to database.

In [2]:
import pandas as pd

# Load the existing DataFrame from CSV
df = pd.read_csv('movie_top_rated.csv')

unique_genres = df['genre'].unique().tolist()

# Extract unique countries from the 'country' column
unique_countries = df['country'].unique().tolist()

# Extract unique actors from the 'actors' column
unique_actors = df['actors'].str.split(', ').explode().unique().tolist()

# Extract unique directors from the 'director' column
unique_directors = df['director'].unique().tolist()

# Create a dictionary to map genres to IDs
genre_dict = {genre: i+1 for i, genre in enumerate(unique_genres)}

# Create a dictionary to map countries to IDs
country_dict = {country: i+1 for i, country in enumerate(unique_countries)}

# Create a dictionary to map actors to IDs
actor_dict = {actor: i+1 for i, actor in enumerate(unique_actors)}

# Create a dictionary to map directors to IDs
director_dict = {director: i+1 for i, director in enumerate(unique_directors)}

# Create a new column 'genre_id' in the DataFrame and assign the corresponding genre ID
df['genre_id'] = df['genre'].apply(lambda x: genre_dict.get(x, 'Unknown'))

# Create a new column 'country_id' in the DataFrame and assign the corresponding country ID
df['country_id'] = df['country'].apply(lambda x: country_dict.get(x, 'Unknown'))

df['actors'] = df['actors'].fillna('')

# Change the format of the "actor_id" column
df['actor_id'] = df['actors'].str.split(', ').apply(lambda x: [str(actor_dict.get(actor, '1000')) for actor in x])

# Create a new column 'director_id' in the DataFrame and assign the corresponding director ID
df['director_id'] = df['director'].apply(lambda x: director_dict.get(x, 'Unknown'))

# Select the desired columns and rearrange them
selected_columns = [
    'id', 'title', 'short_overview', 'overview', 'release_year', 'country_id',
    'rating', 'genre_id', 'actor_id', 'director_id', 'feature', 'kid_restriction',
    'trailer_link', 'movie_link', 'runtime'
]
# Reset the index and rename the new column as 'id' (auto-increment from 1)
df['id'] = df.reset_index().index + 1

# Convert the actor_id column to the desired format ["1", "2", "3"]
df['actor_id'] = df['actor_id'].apply(lambda x: [str(i) for i in x])

df['runtime'] = df['runtime'].astype(int)

df = df[selected_columns]

print(df.shape)

# Export the updated DataFrame to a CSV file
df.to_csv('movie_top_rated.csv', index=False)

# Download the CSV file to your local machine
files.download('movie_top_rated.csv')

(580, 15)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Check the dataframe to see the result.

In [3]:
df.head(10)

,id,title,short_overview,overview,release_year,country_id,rating,genre_id,actor_id,director_id,feature,kid_restriction,trailer_link,movie_link,runtime
0,1,The Godfather,"Spanning the years 1945 to 1955, a chronicle o...","Spanning the years 1945 to 1955, a chronicle o...",1972,1,5,1,"[1, 2, 3]",1,1,0,https://www.youtube.com/watch?v=Ew9ngL1GZvs,http://www.thegodfather.com/,10500
1,2,The Shawshank Redemption,Framed in the 1940s for the double murder of h...,Framed in the 1940s for the double murder of h...,1994,1,5,1,"[4, 5, 6]",2,1,0,https://www.youtube.com/watch?v=PLl99DlL6b4,NaN,8520
2,3,The Godfather Part II,In the continuing saga of the Corleone crime f...,In the continuing saga of the Corleone crime f...,1974,1,5,1,"[2, 7, 8]",1,1,0,https://www.youtube.com/watch?v=rcU6DUUUd7k,NaN,12120
3,4,Dilwale Dulhania Le Jayenge,"Raj is a rich, carefree, happy-go-lucky second...","Raj is a rich, carefree, happy-go-lucky second...",1995,2,5,2,"[9, 10, 11]",3,1,0,https://www.youtube.com/watch?v=oIZ4U21DRlM,NaN,11400
4,5,Schindler's List,The true story of how businessman Oskar Schind...,The true story of how businessman Oskar Schind...,1993,1,5,1,"[12, 13, 14]",4,1,0,https://www.youtube.com/watch?v=mxphAlJID9U,http://www.schindlerslist.com/,11700
5,6,12 Angry Men,The defense and the prosecution have rested an...,The defense and the prosecution have rested an...,1957,1,5,1,"[15, 16, 17]",5,1,0,https://www.youtube.com/watch?v=TEN-2uTi2c0,NaN,5820
6,7,Spirited Away,"A young girl, Chihiro, becomes trapped in a st...","A young girl, Chihiro, becomes trapped in a st...",2001,3,5,3,"[18, 19, 20]",6,1,0,https://www.youtube.com/watch?v=ByXuk9QqQkk,http://movies.disney.com/spirited-away,7500
7,8,Your Name.,High schoolers Mitsuha and Taki are complete s...,High schoolers Mitsuha and Taki are complete s...,2016,3,5,4,"[21, 22, 23]",7,1,0,https://www.youtube.com/watch?v=_oWzYOwXn-o,https://www.funimationfilms.com/movie/yourname/,6360
8,9,Parasite,"All unemployed, Ki-taek's family takes peculia...","All unemployed, Ki-taek's family takes peculia...",2019,4,5,2,"[24, 25, 26]",8,1,0,https://www.youtube.com/watch?v=tBfgTZsBeFM,https://www.parasite-movie.com/,7980
9,10,Spider-Man: Across the Spider-Verse,"After reuniting with Gwen Stacy, Brooklyn’s fu...","After reuniting with Gwen Stacy, Brooklyn’s fu...",2023,1,5,5,"[27, 28, 29]",9,1,0,https://www.youtube.com/watch?v=yFrxzaBLDQM,https://www.acrossthespiderverse.movie,8400


In [4]:
df.dtypes

id                  int64
title              object
short_overview     object
overview           object
release_year        int64
country_id          int64
rating              int64
genre_id            int64
actor_id           object
director_id         int64
feature             int64
kid_restriction     int64
trailer_link       object
movie_link         object
runtime             int64
dtype: object

# Other CSV File

Since the database contain forign key to other tables. We need to seperate it into several csv file to import seperately.

## Table Genre

In [5]:
# Create a new DataFrame with unique genres and genre IDs
unique_genres_df = pd.DataFrame({'genre_id': range(1, len(unique_genres) + 1), 'name': unique_genres})

# Export the unique genres DataFrame to a CSV file
unique_genres_df.to_csv('unique_genres.csv', index=False)

# Download the CSV file to your local machine
files.download('unique_genres.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [9]:
unique_genres_df.head()

,genre_id,name
0,1,Drama
1,2,Comedy
2,3,Animation
3,4,Romance
4,5,Action


## Table Country

In [6]:
# Create a new DataFrame with unique countries and country IDs
unique_countries_df = pd.DataFrame({'country_id': range(1, len(unique_countries) + 1), 'name': unique_countries})

# Export the unique countries DataFrame to a CSV file
unique_countries_df.to_csv('unique_countries.csv', index=False)

# Download the CSV file to your local machine
files.download('unique_countries.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
unique_countries_df.head()

,country_id,name
0,1,United States of America
1,2,India
2,3,Japan
3,4,South Korea
4,5,United Kingdom


## Table Actors

In [7]:
# Create a new DataFrame with unique actors and actor IDs
unique_actors_df = pd.DataFrame({'actor_id': range(1, len(unique_actors) + 1), 'name': unique_actors})

# Export the unique actors DataFrame to a CSV file
unique_actors_df.to_csv('unique_actors.csv', index=False)

# Download the CSV file to your local machine
files.download('unique_actors.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
unique_actors_df.head()

,actor_id,name
0,1,Marlon Brando
1,2,Al Pacino
2,3,James Caan
3,4,Tim Robbins
4,5,Morgan Freeman


## Table Directors

In [8]:
# Create a new DataFrame with unique directors and director IDs
unique_directors_df = pd.DataFrame({'director_id': range(1, len(unique_directors) + 1), 'name': unique_directors})

# Export the unique directors DataFrame to a CSV file
unique_directors_df.to_csv('unique_directors.csv', index=False)

# Download the CSV file to your local machine
files.download('unique_directors.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
unique_directors_df.head()

,director_id,name
0,1,Francis Ford Coppola
1,2,Frank Darabont
2,3,Aditya Chopra
3,4,Steven Spielberg
4,5,Sidney Lumet
